In [15]:
import moxing as mox

mox.file.copy_parallel("runs","obs://lxh-bigdata/runs") 

In [1]:
%pip install torchsummary
%pip install easydict
%pip install ipywidgets

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
You should consider upgrading via the '/home/ma-user/anaconda3/envs/PyTorch-1.8/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
You should consider upgrading via the '/home/ma-user/anaconda3/envs/PyTorch-1.8/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 123 kB 27.2 MB/s eta 0:00:01
     |████████████████████████████████| 73 kB 29.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 115.4 MB/s eta 0:00:01
     |████████████████████████████████| 245 kB 104.0 MB/s eta 0:00:01
     |████████████████████████████████| 72 kB 68.4 MB/s  eta 0:00:01
     |████████████████████████████████| 60 

In [1]:
import sys
sys.path.append("..")
import torch
from torch import nn
from torchvision import transforms
from dataloaders.FashionMnist import load_FashionMnist
from tools.model_trainer import train_model
from tools.model_tester import test_model
from torchsummary import summary
from easydict import EasyDict as edict

# 0 超参数设置

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
cfg = edict({
    'device':device,
    'batch_size': 64,
    'num_classes': 10,  # 分类类别
    'lr': 0.001,  # 学习率
    'epoch_size': 30  # 训练次数
})

# 1 加载数据集

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize((0.1307,), (0.3081,))
])
train_iter, _, test_iter = load_FashionMnist(batch_size=cfg.batch_size, transform=transform)
print(f"train size: {len(train_iter.dataset)}")
# print(f"valid size: {len(valid_iter.dataset)}")
print(f"test size: {len(test_iter.dataset)}")
for X, y in test_iter:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

train size: 60000
test size: 10000
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


# 2 不使用正则化的卷积神经网络

In [4]:
net = nn.Sequential(
    nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=0, bias=False), nn.ReLU(),
    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0, bias=False), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=0, bias=False), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(128*5*5, 120), nn.ReLU(),
    nn.Linear(120, 84), nn.ReLU(),
    nn.Linear(84, cfg.num_classes)
)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=cfg.lr)
train_model(net, train_iter, test_iter, loss_fn, ['accuracy'], optimizer, num_epochs=cfg.epoch_size, device=device, use_tensorboard=True, print_log=True,comment='_DL_Exp2_without_normalization') 

training on cuda
Epoch 1/30
937/937 [===============] Train Loss: 0.5094, Train Accuracy: 0.8118, Valid Loss: 0.3723, Valid Accuracy: 0.8662, Cost Time 2.4782 sec
Epoch 2/30
937/937 [===============] Train Loss: 0.3110, Train Accuracy: 0.8870, Valid Loss: 0.3133, Valid Accuracy: 0.8852, Cost Time 2.5386 sec
Epoch 3/30
937/937 [===============] Train Loss: 0.2560, Train Accuracy: 0.9064, Valid Loss: 0.2784, Valid Accuracy: 0.8973, Cost Time 2.5426 sec
Epoch 4/30
937/937 [===============] Train Loss: 0.2174, Train Accuracy: 0.9201, Valid Loss: 0.2618, Valid Accuracy: 0.9061, Cost Time 2.5020 sec
Epoch 5/30
937/937 [===============] Train Loss: 0.1871, Train Accuracy: 0.9313, Valid Loss: 0.2331, Valid Accuracy: 0.9165, Cost Time 2.4868 sec
Epoch 6/30
937/937 [===============] Train Loss: 0.1590, Train Accuracy: 0.9418, Valid Loss: 0.2450, Valid Accuracy: 0.9122, Cost Time 2.4648 sec
Epoch 7/30
937/937 [===============] Train Loss: 0.1361, Train Accuracy: 0.9495, Valid Loss: 0.2317, Valid 

# 3 对比正则化技术

## 3.1 使用L1正则化

### alpha=1e-5

In [5]:
from tools.model_trainer import train_batch_L1

net = nn.Sequential(
    nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=0, bias=False), nn.ReLU(),
    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0, bias=False), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=0, bias=False), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(128*5*5, 120), nn.ReLU(),
    nn.Linear(120, 84), nn.ReLU(),
    nn.Linear(84, cfg.num_classes)
)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=cfg.lr) 
def train_func(*args, **kwargs):
    return train_batch_L1(*args, **kwargs, lamda=1e-5)

train_model(net, train_iter, test_iter, loss_fn, ['accuracy'], optimizer, num_epochs=cfg.epoch_size, device=device, train_func=train_func, use_tensorboard=True, print_log=True,comment='_DL_Exp2_with_L1_normalization') 

training on cuda
Epoch 1/30
937/937 [===============] Train Loss: 0.5704, Train Accuracy: 0.8103, Valid Loss: 0.3641, Valid Accuracy: 0.8720, Cost Time 3.3293 sec
Epoch 2/30
937/937 [===============] Train Loss: 0.3735, Train Accuracy: 0.8899, Valid Loss: 0.2856, Valid Accuracy: 0.8978, Cost Time 3.2503 sec
Epoch 3/30
937/937 [===============] Train Loss: 0.3296, Train Accuracy: 0.9067, Valid Loss: 0.2596, Valid Accuracy: 0.9052, Cost Time 3.2297 sec
Epoch 4/30
937/937 [===============] Train Loss: 0.2993, Train Accuracy: 0.9181, Valid Loss: 0.2866, Valid Accuracy: 0.8969, Cost Time 3.2090 sec
Epoch 5/30
937/937 [===============] Train Loss: 0.2772, Train Accuracy: 0.9253, Valid Loss: 0.2329, Valid Accuracy: 0.9142, Cost Time 3.3046 sec
Epoch 6/30
937/937 [===============] Train Loss: 0.2592, Train Accuracy: 0.9334, Valid Loss: 0.2325, Valid Accuracy: 0.9161, Cost Time 3.3223 sec
Epoch 7/30
937/937 [===============] Train Loss: 0.2433, Train Accuracy: 0.9420, Valid Loss: 0.2298, Valid 

### alpha=1e-4

In [13]:
from tools.model_trainer import train_batch_L1

net = nn.Sequential(
    nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=0, bias=False), nn.ReLU(),
    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0, bias=False), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=0, bias=False), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(128*5*5, 120), nn.ReLU(),
    nn.Linear(120, 84), nn.ReLU(),
    nn.Linear(84, cfg.num_classes)
)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=cfg.lr) 
def train_func(*args, **kwargs):
    return train_batch_L1(*args, **kwargs, lamda=1e-4)

train_model(net, train_iter, test_iter, loss_fn, ['accuracy'], optimizer, num_epochs=cfg.epoch_size, device=device, train_func=train_func, use_tensorboard=True, print_log=True,comment='_DL_Exp2_with_L1_normalization_1e-4') 

training on cuda
Epoch 1/30
937/937 [===============] Train Loss: 0.7938, Train Accuracy: 0.7794, Valid Loss: 0.4477, Valid Accuracy: 0.8402, Cost Time 3.2000 sec
Epoch 2/30
937/937 [===============] Train Loss: 0.5481, Train Accuracy: 0.8564, Valid Loss: 0.3811, Valid Accuracy: 0.8607, Cost Time 3.1918 sec
Epoch 3/30
937/937 [===============] Train Loss: 0.4821, Train Accuracy: 0.8732, Valid Loss: 0.3683, Valid Accuracy: 0.8670, Cost Time 3.2052 sec
Epoch 4/30
937/937 [===============] Train Loss: 0.4529, Train Accuracy: 0.8828, Valid Loss: 0.3417, Valid Accuracy: 0.8762, Cost Time 3.2120 sec
Epoch 5/30
937/937 [===============] Train Loss: 0.4291, Train Accuracy: 0.8911, Valid Loss: 0.3285, Valid Accuracy: 0.8820, Cost Time 3.2133 sec
Epoch 6/30
937/937 [===============] Train Loss: 0.4109, Train Accuracy: 0.8962, Valid Loss: 0.3278, Valid Accuracy: 0.8783, Cost Time 3.2333 sec
Epoch 7/30
937/937 [===============] Train Loss: 0.3973, Train Accuracy: 0.8987, Valid Loss: 0.2892, Valid 

## 3.2 使用L2正则化

In [6]:
net = nn.Sequential(
    nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=0, bias=False), nn.ReLU(),
    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0, bias=False), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=0, bias=False), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(128*5*5, 120), nn.ReLU(),
    nn.Linear(120, 84), nn.ReLU(),
    nn.Linear(84, cfg.num_classes)
)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=cfg.lr, weight_decay=0.001) # L2正则化
train_model(net, train_iter, test_iter, loss_fn, ['accuracy'], optimizer, num_epochs=cfg.epoch_size, device=device, use_tensorboard=True, print_log=True,comment='_DL_Exp2_with_L2_normalization') 

training on cuda
Epoch 1/30
937/937 [===============] Train Loss: 0.5295, Train Accuracy: 0.8056, Valid Loss: 0.3956, Valid Accuracy: 0.8542, Cost Time 2.5158 sec
Epoch 2/30
937/937 [===============] Train Loss: 0.3527, Train Accuracy: 0.8741, Valid Loss: 0.3395, Valid Accuracy: 0.8801, Cost Time 2.5487 sec
Epoch 3/30
937/937 [===============] Train Loss: 0.3120, Train Accuracy: 0.8875, Valid Loss: 0.3083, Valid Accuracy: 0.8892, Cost Time 2.4886 sec
Epoch 4/30
937/937 [===============] Train Loss: 0.2827, Train Accuracy: 0.8983, Valid Loss: 0.2918, Valid Accuracy: 0.8957, Cost Time 2.5316 sec
Epoch 5/30
937/937 [===============] Train Loss: 0.2661, Train Accuracy: 0.9036, Valid Loss: 0.2854, Valid Accuracy: 0.8949, Cost Time 2.4926 sec
Epoch 6/30
937/937 [===============] Train Loss: 0.2511, Train Accuracy: 0.9097, Valid Loss: 0.2891, Valid Accuracy: 0.8883, Cost Time 2.4708 sec
Epoch 7/30
937/937 [===============] Train Loss: 0.2392, Train Accuracy: 0.9131, Valid Loss: 0.2847, Valid 

## 3.3 使用Dropout正则化

In [7]:
net = nn.Sequential(
    nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=0, bias=False), nn.ReLU(),
    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0, bias=False), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=0, bias=False), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(128*5*5, 120), nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(120, 84), nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(84, cfg.num_classes)
)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=cfg.lr) 
train_model(net, train_iter, test_iter, loss_fn, ['accuracy'], optimizer, num_epochs=cfg.epoch_size, device=device, use_tensorboard=True, print_log=True,comment='_DL_Exp2_with_Dropout') 

training on cuda
Epoch 1/30
937/937 [===============] Train Loss: 0.7192, Train Accuracy: 0.7384, Valid Loss: 0.4254, Valid Accuracy: 0.8432, Cost Time 2.5910 sec
Epoch 2/30
937/937 [===============] Train Loss: 0.4541, Train Accuracy: 0.8445, Valid Loss: 0.3312, Valid Accuracy: 0.8798, Cost Time 2.5365 sec
Epoch 3/30
937/937 [===============] Train Loss: 0.3813, Train Accuracy: 0.8699, Valid Loss: 0.2914, Valid Accuracy: 0.8939, Cost Time 2.5714 sec
Epoch 4/30
937/937 [===============] Train Loss: 0.3399, Train Accuracy: 0.8849, Valid Loss: 0.2843, Valid Accuracy: 0.8960, Cost Time 2.5497 sec
Epoch 5/30
937/937 [===============] Train Loss: 0.3081, Train Accuracy: 0.8947, Valid Loss: 0.2591, Valid Accuracy: 0.9075, Cost Time 2.5392 sec
Epoch 6/30
937/937 [===============] Train Loss: 0.2834, Train Accuracy: 0.9034, Valid Loss: 0.2491, Valid Accuracy: 0.9086, Cost Time 2.5254 sec
Epoch 7/30
937/937 [===============] Train Loss: 0.2651, Train Accuracy: 0.9081, Valid Loss: 0.2497, Valid 

## 3.4 使用BN正则化

In [8]:
net = nn.Sequential(
    nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=0, bias=False),
    nn.BatchNorm2d(32), nn.ReLU(),
    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0, bias=False),
    nn.BatchNorm2d(64), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=0, bias=False),
    nn.BatchNorm2d(128), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(128*5*5, 120), nn.ReLU(),
    nn.Linear(120, 84), nn.ReLU(),
    nn.Linear(84, cfg.num_classes)
)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=cfg.lr) 
train_model(net, train_iter, test_iter, loss_fn, ['accuracy'], optimizer, num_epochs=cfg.epoch_size, device=device, use_tensorboard=True, print_log=True,comment='_DL_Exp2_with_BN') 

training on cuda
Epoch 1/30
937/937 [===============] Train Loss: 0.3727, Train Accuracy: 0.8644, Valid Loss: 0.3249, Valid Accuracy: 0.8812, Cost Time 3.2780 sec
Epoch 2/30
937/937 [===============] Train Loss: 0.2392, Train Accuracy: 0.9132, Valid Loss: 0.2530, Valid Accuracy: 0.9057, Cost Time 3.2588 sec
Epoch 3/30
937/937 [===============] Train Loss: 0.1973, Train Accuracy: 0.9275, Valid Loss: 0.2265, Valid Accuracy: 0.9148, Cost Time 3.2807 sec
Epoch 4/30
937/937 [===============] Train Loss: 0.1682, Train Accuracy: 0.9389, Valid Loss: 0.2439, Valid Accuracy: 0.9115, Cost Time 3.2823 sec
Epoch 5/30
937/937 [===============] Train Loss: 0.1416, Train Accuracy: 0.9485, Valid Loss: 0.2427, Valid Accuracy: 0.9154, Cost Time 3.2880 sec
Epoch 6/30
937/937 [===============] Train Loss: 0.1205, Train Accuracy: 0.9562, Valid Loss: 0.2338, Valid Accuracy: 0.9213, Cost Time 3.3667 sec
Epoch 7/30
937/937 [===============] Train Loss: 0.1039, Train Accuracy: 0.9616, Valid Loss: 0.2401, Valid 

## 3.5 使用Layer Normalization正则化

In [5]:
summary(net, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             288
              ReLU-2           [-1, 32, 26, 26]               0
            Conv2d-3           [-1, 64, 24, 24]          18,432
              ReLU-4           [-1, 64, 24, 24]               0
         MaxPool2d-5           [-1, 64, 12, 12]               0
            Conv2d-6          [-1, 128, 10, 10]          73,728
              ReLU-7          [-1, 128, 10, 10]               0
         MaxPool2d-8            [-1, 128, 5, 5]               0
           Flatten-9                 [-1, 3200]               0
           Linear-10                  [-1, 120]         384,120
             ReLU-11                  [-1, 120]               0
           Linear-12                   [-1, 84]          10,164
             ReLU-13                   [-1, 84]               0
           Linear-14                   

In [9]:
net = nn.Sequential(
    nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=0, bias=False),
    nn.LayerNorm((32, 26, 26)), nn.ReLU(),
    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0, bias=False),
    nn.LayerNorm((64, 24, 24)), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=0, bias=False),
    nn.LayerNorm((128, 10, 10)), nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(128*5*5, 120), nn.ReLU(),
    nn.Linear(120, 84), nn.ReLU(),
    nn.Linear(84, cfg.num_classes)
)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=cfg.lr) 
train_model(net, train_iter, test_iter, loss_fn, ['accuracy'], optimizer, num_epochs=cfg.epoch_size, device=device, use_tensorboard=True, print_log=True,comment='_DL_Exp2_with_LN') 

training on cuda
Epoch 1/30
937/937 [===============] Train Loss: 0.4131, Train Accuracy: 0.8475, Valid Loss: 0.2990, Valid Accuracy: 0.8871, Cost Time 3.2308 sec
Epoch 2/30
937/937 [===============] Train Loss: 0.2566, Train Accuracy: 0.9053, Valid Loss: 0.2691, Valid Accuracy: 0.9018, Cost Time 3.1987 sec
Epoch 3/30
937/937 [===============] Train Loss: 0.2173, Train Accuracy: 0.9196, Valid Loss: 0.2423, Valid Accuracy: 0.9120, Cost Time 3.1693 sec
Epoch 4/30
937/937 [===============] Train Loss: 0.1888, Train Accuracy: 0.9308, Valid Loss: 0.2423, Valid Accuracy: 0.9171, Cost Time 3.1777 sec
Epoch 5/30
937/937 [===============] Train Loss: 0.1670, Train Accuracy: 0.9382, Valid Loss: 0.2272, Valid Accuracy: 0.9174, Cost Time 3.1716 sec
Epoch 6/30
937/937 [===============] Train Loss: 0.1473, Train Accuracy: 0.9456, Valid Loss: 0.2289, Valid Accuracy: 0.9211, Cost Time 3.1414 sec
Epoch 7/30
937/937 [===============] Train Loss: 0.1303, Train Accuracy: 0.9518, Valid Loss: 0.2473, Valid 

## 3.6 SELU & alpha dropout

In [10]:
net = nn.Sequential(
    nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=0, bias=False), nn.SELU(),
    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0, bias=False), nn.SELU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=0, bias=False), nn.SELU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(128*5*5, 120), nn.SELU(),
    nn.Linear(120, 84), nn.SELU(),
    nn.Linear(84, cfg.num_classes)
)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=cfg.lr)
train_model(net, train_iter, test_iter, loss_fn, ['accuracy'], optimizer, num_epochs=cfg.epoch_size, device=device, use_tensorboard=True, print_log=True,comment='_DL_Exp2_with_SELU') 

training on cuda
Epoch 1/30
937/937 [===============] Train Loss: 0.4100, Train Accuracy: 0.8482, Valid Loss: 0.3161, Valid Accuracy: 0.8805, Cost Time 2.4417 sec
Epoch 2/30
937/937 [===============] Train Loss: 0.2682, Train Accuracy: 0.9017, Valid Loss: 0.2884, Valid Accuracy: 0.8933, Cost Time 2.4543 sec
Epoch 3/30
937/937 [===============] Train Loss: 0.2177, Train Accuracy: 0.9182, Valid Loss: 0.2503, Valid Accuracy: 0.9081, Cost Time 2.5150 sec
Epoch 4/30
937/937 [===============] Train Loss: 0.1809, Train Accuracy: 0.9318, Valid Loss: 0.2608, Valid Accuracy: 0.9073, Cost Time 2.4628 sec
Epoch 5/30
937/937 [===============] Train Loss: 0.1523, Train Accuracy: 0.9432, Valid Loss: 0.2738, Valid Accuracy: 0.9123, Cost Time 2.4787 sec
Epoch 6/30
937/937 [===============] Train Loss: 0.1280, Train Accuracy: 0.9527, Valid Loss: 0.2637, Valid Accuracy: 0.9133, Cost Time 2.4574 sec
Epoch 7/30
937/937 [===============] Train Loss: 0.1076, Train Accuracy: 0.9600, Valid Loss: 0.3196, Valid 

In [11]:
net = nn.Sequential(
    nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=0, bias=False), nn.SELU(),
    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0, bias=False), nn.SELU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=0, bias=False), nn.SELU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(128*5*5, 120), nn.SELU(),
    nn.AlphaDropout(0.5),
    nn.Linear(120, 84), nn.SELU(),
    nn.AlphaDropout(0.5),
    nn.Linear(84, cfg.num_classes)
)


loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=cfg.lr)
train_model(net, train_iter, test_iter, loss_fn, ['accuracy'], optimizer, num_epochs=cfg.epoch_size, device=device, use_tensorboard=True, print_log=True,comment='_DL_Exp2_with_SELU_alphaDropout') 

training on cuda
Epoch 1/30
937/937 [===============] Train Loss: 0.6233, Train Accuracy: 0.7721, Valid Loss: 0.8246, Valid Accuracy: 0.8537, Cost Time 2.7234 sec
Epoch 2/30
937/937 [===============] Train Loss: 0.4090, Train Accuracy: 0.8575, Valid Loss: 0.7784, Valid Accuracy: 0.8860, Cost Time 2.6667 sec
Epoch 3/30
937/937 [===============] Train Loss: 0.3524, Train Accuracy: 0.8774, Valid Loss: 0.6248, Valid Accuracy: 0.8953, Cost Time 2.6639 sec
Epoch 4/30
937/937 [===============] Train Loss: 0.3165, Train Accuracy: 0.8898, Valid Loss: 0.7563, Valid Accuracy: 0.8996, Cost Time 2.7380 sec
Epoch 5/30
937/937 [===============] Train Loss: 0.2942, Train Accuracy: 0.8967, Valid Loss: 0.6640, Valid Accuracy: 0.9011, Cost Time 2.7181 sec
Epoch 6/30
937/937 [===============] Train Loss: 0.2742, Train Accuracy: 0.9036, Valid Loss: 0.6616, Valid Accuracy: 0.9109, Cost Time 2.7022 sec
Epoch 7/30
937/937 [===============] Train Loss: 0.2572, Train Accuracy: 0.9110, Valid Loss: 0.5880, Valid 